In [ ]:
# stdlib
# import os

# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"

In [ ]:
# use_live_bigquery = False
# os.environ["TEST_BIGQUERY_APIS_LIVE"] = str(use_live_bigquery)

In [ ]:
# isort: off
# stdlib
from collections import Counter

import os
import sys

# syft absolute
import syft as sy
from syft.util.util import (
    find_base_dir_with_tox_ini,
    get_caller_file_path,
    is_interpreter_jupyter,
)


def add_helper_path_to_python_path() -> None:
    current_path = "."

    # jupyter uses "." which resolves to the notebook
    if not is_interpreter_jupyter():
        # python uses the file which has from syft import test_settings in it
        import_path = get_caller_file_path()
        if import_path:
            current_path = import_path

    base_dir = find_base_dir_with_tox_ini(current_path)
    notebook_helper_path = os.path.join(
        base_dir, "notebooks/scenarios/bigquery/upgradability/0.9.1_helpers"
    )
    sys.path.append(notebook_helper_path)


add_helper_path_to_python_path()
from syft.service.job.job_stash import JobStatus

# third party
from email_helpers import get_email_server

# isort: on
# third party
from job_helpers import TestJob
from job_helpers import create_jobs
from job_helpers import extract_code_path
from job_helpers import save_jobs

In [ ]:
environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
environment

In [ ]:
SERVER_PORT = "8080"
SERVER_URL = f"http://localhost:{SERVER_PORT}"
ADMIN_EMAIL, ADMIN_PW = "admin2@bigquery.org", "bqpw2"

# Launch server & login

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high-migrations",
    dev_mode=True,
    server_side_type="high",
    port=SERVER_PORT,
    n_consumers=4,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
email_server, smtp_server = get_email_server(reset=True)

In [ ]:
high_client = sy.login(
    url="http://localhost:8080", email=ADMIN_EMAIL, password=ADMIN_PW
)

In [ ]:
# third party
from email_helpers import load_users

In [ ]:
[user.email for user in high_client.users.get_all()]

In [ ]:
users = load_users(high_client)

# Create jobs

# Inspect job data (requests for these jobs to be created)

In [ ]:
num_jobs = int(os.environ.get("NUM_TEST_JOBS", 10))

jobs_data = create_jobs(users, total_jobs=num_jobs)

In [ ]:
counts = Counter([j.job_type for j in jobs_data])
for k, v in counts.most_common():
    print(f"{k}: #{v}")

In [ ]:
print(f"{len(jobs_data)=}")

for job in jobs_data:
    print(f"{job.job_type=}, {job.should_succeed=}, {job.should_submit=}")

In [ ]:
assert len(jobs_data) == num_jobs
assert all(isinstance(j, TestJob) for j in jobs_data)
assert all(job.client is not None for job in jobs_data)

In [ ]:
save_jobs(jobs_data)

# Submit jobs


In [ ]:
admin_emails_before = len(email_server.get_emails_for_user("admin@bigquery.org"))
print(f"{admin_emails_before=}")

## Test Succesful jobs

In [ ]:
jobs_submit_should_succeed = [j for j in jobs_data if j.should_submit]

In [ ]:
for job in jobs_submit_should_succeed:
    client = job.client
    response = client.api.services.bigquery.submit_query(
        func_name=job.func_name, query=job.query
    )
    job.code_path = extract_code_path(response)

## Test failures

In [ ]:
jobs_submit_should_fail = [j for j in jobs_data if not j.should_submit]

In [ ]:
for job in jobs_submit_should_fail:
    client = job.client

    with sy.raises(sy.SyftException):
        client.api.services.bigquery.submit_query(
            func_name=job.func_name, query=job.query
        )

In [ ]:
for job in jobs_data:
    print(f"Job {job.func_name:.20} {job.should_submit=}, {job.is_submitted=}")

assert all(job.is_submitted == job.should_submit for job in jobs_data)

In [ ]:
save_jobs(jobs_data)

## Test: cannot execute submitted jobs yet

In [ ]:
submitted_jobs = [job for job in jobs_data if job.should_submit]
job_execution_fns = [getattr(job.client.code, job.code_path) for job in submitted_jobs]
assert len(submitted_jobs)  # failsafe for next tests

In [ ]:
for fn in job_execution_fns:
    # blocking
    with sy.raises(
        sy.SyftException(public_message="*Your code is waiting for approval*")
    ):
        result = fn()

In [ ]:
for fn in job_execution_fns:
    # nonblocking
    result_job = fn(blocking=False)
    result_job.wait()
    assert isinstance(result_job.result, sy.SyftError)
    assert result_job.status == JobStatus.ERRORED

# Verify that admin has emails for submitted requests

In [ ]:
num_should_submit = sum(j.should_submit for j in jobs_data)
admin_emails_after = len(email_server.get_emails_for_user("admin@bigquery.org"))
print("admin emails after", admin_emails_after)
assert admin_emails_after >= admin_emails_before + num_should_submit
# assert len(users_emails) > after_number_of_emails
# assert len(users_emails) == after_number_of_emails + 1

# Cleanup

In [ ]:
smtp_server.stop()

In [ ]:
server.land()